In [ ]:
"""
Purpose: To check on the status of the Allen split suggestions pipeline
as they run through

- Downloading
- Decimation
- Soma Finding
- Decomposition
- Suggestion Splits

"""

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty/")

In [3]:
from importlib import reload

import neuron_utils as nru

import neuron

import neuron_visualizations as nviz

import time

import datajoint_utils as du

import numpy as np

import proofreading_utils as pru

import preprocessing_vp2 as pre

# -- For the axon classification --

import neuron_searching as ns

import skeleton_utils as sk

import numpy_utils as nu

import networkx_utils as xu

import system_utils as su

INFO - 2021-03-22 18:47:18,315 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-22 18:47:18,318 - settings - Setting database.user to celiib
INFO - 2021-03-22 18:47:18,319 - settings - Setting database.password to newceliipass
INFO - 2021-03-22 18:47:18,334 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-03-22 18:47:18,335 - settings - Setting enable_python_native_blobs to True
INFO - 2021-03-22 18:47:18,349 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-03-22 18:47:18,771 - settings - Setting enable_python_native_blobs to True
INFO - 2021-03-22 18:47:18,775 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-22 18:47:18,776 - settings - Setting database.user to celiib
INFO - 2021-03-22 18:47:18,777 - settings - Setting database.password to newceliipass
INFO - 2021-03-22 18:47:18,778 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-22 18:47:18,779 - settings - Setting database.user to celiib
INFO - 2021-03-22 18:47:18,779 - settings - Setting database.password to newceliipass
INFO - 2021-03-22 18:47:18,782 - settings - Setting enable_python_native_blobs to True
INFO - 2021-03-22 18:47:19,233 - settings - Setting enable_python_native_blobs to True


In [4]:
minnie,schema = du.configure_minnie_vm()

INFO - 2021-03-22 18:47:21,551 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-03-22 18:47:21,552 - settings - Setting database.user to celiib
INFO - 2021-03-22 18:47:21,553 - settings - Setting database.password to newceliipass
INFO - 2021-03-22 18:47:21,557 - settings - Setting enable_python_native_blobs to True
INFO - 2021-03-22 18:47:22,040 - settings - Setting enable_python_native_blobs to True


# Investigating why some neurons did not get split suggestions

In [ ]:
allen_table = (minnie.AllenProofreading() & dict(month=3,day=18,year=2021))
allen_table

In [ ]:
minnie.BaylorSegmentCentroid() & allen_table  & dict(decimation_ratio=0.25) & "multiplicity=0"

In [ ]:
nviz.plot_objects(du.fetch_segment_id_mesh(864691136236789519))

In [ ]:
key_source = minnie.Decomposition() & (minnie.AllenProofreading() & dict(month=3,day=3,year=2021)).proj()
key_source

In [ ]:
key_source & "n_multi_soma_touching_limbs = 0 AND n_same_soma_multi_touching_limbs=0 "

In [ ]:
(minnie.Decomposition() & dict(segment_id=864691136008729518))

In [ ]:
neuron_obj = (minnie.Decomposition() & dict(segment_id=864691136008729518)).fetch1("decomposition")
neuron_obj

In [ ]:
import datajoint_utils as du
neuron_obj = du.fetch_neuron_obj_manual(segment_id=864691136008729518)

In [ ]:
import neuron_utils as nru

In [ ]:
segment_id = 864691135782479155
dec_mesh = du.fetch_segment_id_mesh(segment_id)
neuron_obj = nru.decompress_neuron("/mnt/dj-stor01/platinum/minnie65/02/decomposition/864691135782479155_0_25.pbz2",
                     original_mesh=dec_mesh)

In [ ]:
nviz.visualize_neuron(neuron_obj,
                     limb_branch_dict="all")

In [ ]:
neuron_obj.n_error_limbs

In [ ]:
nviz.visualize_neuron_lite(neuron_obj)

In [ ]:
segment_id_with_no_errors = [
    864691135345291423, #had just soma on the very tip
    864691135373423816, #removed in glia
    864691135753669837,
    ,
]

In [ ]:
curr_table = (minnie.schema.jobs & "table_name='__decomposition'") & "status='error'"
(curr_table).delete()# & "status='error'")

In [ ]:
curr_table = (minnie.schema.jobs & "table_name='__decomposition_split'")
seg_ids_to_delete = [k["segment_id"] for k in curr_table.fetch("key")]
seg_ids_to_delete

In [ ]:
minnie.Decomposition()

# All the cells that need to get suggestions

In [ ]:
key_source = minnie.Decomposition() & "n_somas>1 OR n_error_limbs>0" & (minnie.AllenProofreading() & dict(month=3,day=3,year=2021)).proj()
key_source

In [ ]:
no_splits = (minnie.AllenProofreading() & dict(month=3,day=3,year=2021)) - key_source.proj()
no_splits

In [ ]:
minnie.DecompositionSplit() & key_source.proj()

In [ ]:
curr_table = (minnie.schema.jobs & "table_name='__decomposition_split'")
(curr_table)#.delete()# & "status='error'")

# Getting the split suggesstions

In [ ]:
import datajoint as dj
dj.U("segment_id") & minnie.DecompositionSplit() & key_source.proj()

In [ ]:
du.create_suggested_splits_neuroglancer_spreadsheet?

In [5]:
minnie.AllenProofreading() 

segment_id segment id for those to be decimated,month the month that the cell was added,day the day that the cell was added,year the year that the cell was added,n_somas the number of somas associated with the neuron from the Allen Institute
864691134964412191,1,28,2021,2
864691134988608634,1,28,2021,2
864691134988608634,3,3,2021,2
864691135012398582,1,28,2021,2
864691135065024068,1,28,2021,2
864691135065024068,3,3,2021,2
864691135065024068,3,11,2021,2
864691135081756919,3,11,2021,3
864691135081756919,3,18,2021,3
864691135132762784,3,18,2021,3


In [7]:
allen_segment_ids = (minnie.AllenProofreading() & dict(day=18,month=3,year=2021)).fetch("segment_id")
allen_segment_ids

array([864691135081756919, 864691135132762784, 864691135337764326,
       864691135340990917, 864691135341029829, 864691135345208735,
       864691135345280671, 864691135373423816, 864691135382504922,
       864691135382719450, 864691135403793646, 864691135415490234,
       864691135462292893, 864691135462301853, 864691135462454941,
       864691135462553757, 864691135474751296, 864691135491253343,
       864691135497625875, 864691135501584194, 864691135502074805,
       864691135538111858, 864691135583731320, 864691135589222386,
       864691135590161419, 864691135617824143, 864691135654122946,
       864691135724471467, 864691135724472235, 864691135730176441,
       864691135736940420, 864691135748581673, 864691135753746381,
       864691135777347517, 864691135865548421, 864691135937302020,
       864691135937310468, 864691136023802169, 864691136041089110,
       864691136089091639, 864691136194333772, 864691136209271740,
       864691136236789519, 864691136237486908, 864691136286740

In [8]:
len(allen_segment_ids)

57

In [10]:
allen_segment_ids = (minnie.AllenProofreading() & dict(day=18,month=3,year=2021)).fetch("segment_id")
du.create_suggested_splits_neuroglancer_spreadsheet(segment_ids=allen_segment_ids,output_type="server",
                                                   print_links=True,
                                                   output_filename="allen_spreadsheet_3_18.csv")

  0%|          | 0/44 [00:00<?, ?it/s]

segment_id = 864691135081756919
segment_id = 864691135132762784
segment_id = 864691135337764326
segment_id = 864691135340990917
segment_id = 864691135341029829
segment_id = 864691135345208735
segment_id = 864691135382504922
segment_id = 864691135382719450
segment_id = 864691135403793646
segment_id = 864691135415490234
segment_id = 864691135462292893
segment_id = 864691135462301853
segment_id = 864691135462454941
segment_id = 864691135462553757
segment_id = 864691135474751296
segment_id = 864691135497625875
segment_id = 864691135501584194
segment_id = 864691135502074805
segment_id = 864691135538111858
segment_id = 864691135590161419
segment_id = 864691135617824143
segment_id = 864691135654122946
segment_id = 864691135736940420
segment_id = 864691135748581673
segment_id = 864691135753746381
segment_id = 864691135777347517
segment_id = 864691135865548421
segment_id = 864691135937302020
segment_id = 864691135937310468
segment_id = 864691136023802169
segment_id = 864691136041089110
segment_

In [11]:
du.fetch_neuron_obj_manual(864691135753746381)

Decompressing Neuron in minimal output mode...please wait


In [12]:
neuron_obj = Out[11]

In [14]:
import neuron_visualizations as nviz
nviz.visualize_neuron(neuron_obj,
                     limb_branch_dict="all")


 Working on visualization type: mesh

 Working on visualization type: skeleton
